## Carregando os dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import scatter_matrix

#importando o arquivo
df = pd.read_csv('/content/drive/My Drive/Datasets/iris2.csv', sep=',')
df.head()


## Pré-processamento: eliminação de atributos irrelevantes

In [ ]:
# remove as colunas id_planta e cidade_origem
df = df.drop(columns=['id_planta','cidade_origem'])

# imprime o dataframe
display(df.head(n=10))

## Pré-processamento: tratamento de atributos com valores ausentes

In [ ]:
# índices das linhas que contém valores NaN
idxRowNan = pd.isnull(df).any(axis=1).to_numpy().nonzero()

# imprime apenas as linhas com valoes ausentes
display(df.iloc[idxRowNan])

In [ ]:
def trataFaltantes(df):
    '''
    Substitui os valores faltantes pela média dos outros valores do mesmo atributo
    de amostras que sejam da mesma classe
    '''

    # seleciona apenas as linhas da base de dados onde a coluna largura_sepala não contém valores nulos
    notNull_ls = df.loc[ ~pd.isnull(df['largura_sepala']), :]
    notNull_cp = df.loc[ ~pd.isnull(df['comprimento_petala']), :]

    # calcula a media dos valores do atributo largura_sepala que não são nulos e que são da classe Iris-setosa
    media_ls = notNull_ls[ notNull_ls['classe']=='Iris-setosa' ]['largura_sepala'].mean()
    media_cp = notNull_cp[ notNull_cp['classe']=='Iris-setosa' ]['comprimento_petala'].mean()

    # substitui os valores nulos pela média
    df.loc[ pd.isnull(df['largura_sepala']), 'largura_sepala'] = media_ls
    df.loc[ pd.isnull(df['comprimento_petala']), 'comprimento_petala'] = media_cp

    return df

trataFaltantes(df)

# imprime apenas as linhas que antes possuiam valores NaN
print('\nAmostras que possuiam valores faltantes:')
display(df.iloc[idxRowNan])

## Pré-processamento: tratamento de dados inconsistentes ou redundantes

In [ ]:
df_duplicates = df[ df.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)]

# se houver valores redundantes ou inconsistentes, imprima
if len(df_duplicates)>0:
    print('\nAmostras redundantes ou inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem valores duplicados')

In [ ]:
def delDuplicatas(df):
   '''
   Para cada grupo de amostras duplicadas, mantém uma e apaga as demais
   '''

   # remove as amostras duplicadas, mantendo apenas a primeira ocorrencia
   df = df.drop_duplicates(keep = 'first')

   return df

df = delDuplicatas(df)

display(df)


In [ ]:
# para detectar inconsistências, a rotina abaixo obtém as amostras onde os valores
# dos atributos continuam duplicados. Neste caso, os atributos serão iguais, mas as classes serão distintas
df_duplicates = df[ df.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)]

# se tiver valores inconsistentes, imprime
if len(df_duplicates)>0:
   print('\nAmostras inconsistentes:')
   display(df_duplicates)
else:
   print('Não existem mostras inconsistentes')


In [ ]:
def delInconsistencias(df):
   '''
   Remove todas as amostras inconsistentes da base de dados
   '''

   df = df.drop_duplicates(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'], keep = False)
   return df

df = delInconsistencias(df)

# obtém apenas as amostras onde os valores dos atributos estão duplicados
df_duplicates = df[ df.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)]

# se tiver valores redundantes ou inconsistentes, imprime
if len(df_duplicates)>0:
   display(df_duplicates)
else:
   print('Não existem amostras redundantes ou inconsistentes')

In [ ]:
# apresenta as principais estatísticas da base de dados
df_detalhes = df.describe()

display(df_detalhes)


## Pré-processamento: normalização dos atributos

In [ ]:
def normalizar(X):
   """
   Normaliza os atributos em X

   Esta função retorna uma versao normalizada de X onde o valor da
   média de cada atributo é igual a 0 e desvio padrao é igual a 1. Trata-se de
   um importante passo de pré-processamento quando trabalha-se com
   métodos de aprendizado de máquina.
   """

   m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto

   # Incializa as variaves de saída
   X_norm = np.random.rand(m,n) # inicializa X_norm com valores aleatórios
   mu = 0 # inicializa a média
   sigma = 1 # inicializa o desvio padrão

   mu = np.mean(X, axis=0)
   sigma = np.std(X, axis=0, ddof=1)

   for i in range(m):
       X_norm[i,:] = (X[i,:]-mu) / sigma


   return X_norm, mu, sigma


# coloca os valores dos atributos na variável X
X = df.iloc[:,0:-1].values

# chama a função para normalizar X
X_norm, mu, sigma = normalizar(X)

df.iloc[:,0:-1] = X_norm

print('\nPrimeira amostra da base antes da normalização: [%2.4f %2.4f].' %(X[0,0],X[0,1]))
print('\nApós a normalização, espera-se que a primeira amostra seja igual a: [-0.5747 0.1804].')
print('\nPrimeira amostra da base após normalização: [%2.4f %2.4f].' %(X_norm[0,0],X_norm[0,1]))



In [ ]:
# apresenta as principais estatísticas da base de dados
df_detalhes = df.describe()

display(df_detalhes.round(8))

## Pré-processamento: detecção de *outliers*

In [ ]:
# gera um bloxplot para cada atributo
df.boxplot(figsize=(15,7))
plt.show()


In [ ]:
pd.plotting.scatter_matrix(df, figsize=(18,18))
plt.show()


In [ ]:
# matriz de gráficos scatter
sns.pairplot(df, hue='classe', height=3.5);

# mostra o gráfico usando a função show() da matplotlib
plt.show()


In [ ]:
# define o scatter plot
sns.lmplot(x='comprimento_sepala', y='largura_sepala', data=df,
          fit_reg=False,
          hue='classe')

# cria um título para o gráfico
plt.title('Comprimento vs largura da sepala')

# mostra o gráfico
plt.show()


In [ ]:
for atributo in df.columns[:-1]:
   # define a dimensão do gráfico
   plt.figure(figsize=(8,8))

   # cria o boxplot
   sns.boxplot(x="classe", y=atributo, data=df, whis=1.5)

   # mostra o gráfico
   plt.show()


In [ ]:
def removeOutliers(df):
   """
   Remove os outliers da base de dados
   """

   for atributo in df.columns[:-1]:

       # obtem o terceiro e o primeiro quartil.
       q75, q25 = np.percentile(df[atributo].values, [75 ,25])

       # calcula o IQR
       IQR = q75 - q25

       # remove os outliers com base no valor do IQR
       df = df[ (df[atributo]<=(q75+1.5*IQR)) & (df[atributo]>=(q25-1.5*IQR)) ]

   return df

# remove os outliers
df = removeOutliers(df)

# apresenta as principais estatísticas sobre a base de dados
df.boxplot(figsize=(15,7))
plt.show()

# matriz de gráficos scatter
sns.pairplot(df, hue='classe', height=3.5);

# mostra o gráfico usando a função show() da matplotlib
plt.show()



## Pré-processamento: distribuição das classes

In [ ]:
display(df['classe'].value_counts())

# cria um gráfico de barras com a frequência de cada classe
sns.countplot(x="classe", data=df)

# mostra o gráfico
plt.show()

## Pré-processamento: correlação entre os atributos

In [ ]:
# criando uma matriz X com os valores do data frame
X = df.iloc[:,:-1].values

# matriz de correlação
correlation = np.corrcoef(X, rowvar=False)

print('\n\nMatriz de correlação: ')
display(correlation)

In [ ]:
# matriz de correlação
df_correlation = df.corr()

print('\n\nMatriz de correlação: ')
display(df_correlation)

In [ ]:
# cria um mapa de cores dos valores da correlação
sns.heatmap(df_correlation,
        xticklabels=df_correlation.columns,
        yticklabels=df_correlation.columns)

plt.title('Correlacao')
plt.show()